In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint
import pandas as pd
import numpy as np
import time
import timeit

# start program run timer
start = timeit.default_timer()

In [2]:
# Activate browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 92.0.4515
[WDM] - Get LATEST driver version for 92.0.4515


[WDM] - Driver [/Users/Kaboni/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


In [3]:
# Path for URL(mens)
mens = '/leaderboard/games/2021?division=1&sort=0'
url = 'https://games.crossfit.com'
webpage = f'{url}{mens}'
browser.visit(webpage)

In [4]:
# Accept cookies
browser.find_by_xpath('//*[@id="pp-cookie-accept"]').click()

# Close on load pop-up
browser.find_by_xpath('//*[@class="mc-closeModal"]').click()

# Move table to event 1
leftArrow = browser.find_by_xpath('//button[starts-with(@class,"workout")]')
rightArrow = browser.find_by_xpath('//button[contains(@class,"right")]')
leftArrow.click()
leftArrow.click()

In [5]:
# Retrieve elements (mens)

# defining page right arrow
rightArrow = browser.find_by_xpath('//button[contains(@class,"right")]')
expand = browser.find_by_xpath('//*[contains(@class,"expand")]')

# initializing lists
overall = []
individual = []
x = 0
y = 0

# find table row to iterate through
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
articles = soup.find('tbody')
athletes = articles.find_all('tr', class_='collapsed')

# for loop to find men's info
for i in athletes:
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find('tbody')
    athletes = articles.find_all('tr', class_='collapsed')
    athleteJustinM = athletes[x]
    fName = athleteJustinM.find('div', class_='first-name').text.strip()
    individual.append(fName)
    lName = athleteJustinM.find('div', class_='last-name').text.strip()
    individual.append(lName)
    #new
    fullName = f'{fName} {lName}'
    individual.append(fullName)
    a = athleteJustinM.find('a')
    href = a.get('href')
    
    webpage = f'{url}{href}'
    browser.visit(webpage)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    body = soup.find('body')
    conatinerOverlay = body.find('div', class_='container-overlay')
    mainContent = conatinerOverlay.find('div', class_='main-content')

    md9 = mainContent.find('div', class_='col-md-9')
    img = md9.find('img')
    picture = img.get('src')

    pageCover = mainContent.find('div', class_='page-cover')
    
    overlayed = pageCover.find('div', class_='overlayed')
    clearfix = overlayed.find('ul')
    allLi = clearfix.find_all('li')
    selectedLi = allLi[y]
    flagIMG = selectedLi.find('img')
    flag = flagIMG.get('src')
    flagURL = f'https:{flag}'

    selectedLi = allLi[y]
    countryDiv = selectedLi.find('div', class_='text')
    country = countryDiv.find('small').text

    thirdRow = clearfix.find('li', class_='third-row')
    gymText = thirdRow.find('div', class_='text')
    try:
        affiliate = gymText.find('a').text
    except:
        affiliate = ""

    individual.append(picture)
    individual.append(country)
    individual.append(flagURL)
    individual.append(affiliate)

    browser.back()
    rightArrow = browser.find_by_xpath('//button[contains(@class,"right")]')
    leftArrow = browser.find_by_xpath('//button[starts-with(@class,"workout")]')
    leftArrow.click()
    leftArrow.click()
    #new fin
    scores = athleteJustinM.find_all('span', class_='result')
    try:
        individual.append(scores[0].text.strip())
        individual.append(scores[1].text.strip())
        individual.append(scores[2].text.strip())
        individual.append(scores[3].text.strip())
        individual.append(scores[4].text.strip())
        individual.append(scores[5].text.strip())
    except:
        print(fName, "no score")
        pass

    rightArrow.click()

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find('tbody')
    athletes = articles.find_all('tr', class_='collapsed')
    athleteJustinM = athletes[x]
    scores = athleteJustinM.find_all('span', class_='result')
    try:
        individual.append(scores[0].text.strip())
        individual.append(scores[1].text.strip())
        individual.append(scores[2].text.strip())
        individual.append(scores[3].text.strip())
        individual.append(scores[4].text.strip())
        individual.append(scores[5].text.strip())
    except:
        print(fName, "no score 2")
        pass

    rightArrow.click()

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find('tbody')
    athletes = articles.find_all('tr', class_='collapsed')
    athleteJustinM = athletes[x]
    scores = athleteJustinM.find_all('span', class_='result')
    try:
        individual.append(scores[3].text.strip())
        individual.append(scores[4].text.strip())
        individual.append(scores[5].text.strip())
    except:
        print(fName, "no score 3")
        pass

    overall.append(individual)
    leftArrow.click()
    leftArrow.click()
    x+=1
    time.sleep(.4)
    break

Roman no score
Roman no score 2
Roman no score 3
Stas no score
Stas no score 2
Stas no score 3


In [6]:
# pulling the 38 male athletes that competed (40 made it, 2 withdrew prior to event)
k = 0
s = 22
allAthletes = []
for i in individual:
    allAthletes.append(individual[k:s])
    k+=22
    s+=22
    
athlete_list = allAthletes[:38]
# athlete_list

In [7]:
# putting male athletes into a dataframe
full_df = pd.DataFrame(athlete_list)

In [8]:
# finding which division the women are in to append the URL
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
body = soup.find('body')
container = body.find('div', class_='container-overlay')
mainContent = container.find('div', class_='main-content')
lbMain = mainContent.find('div', class_='lb-main')
innerDesktop = lbMain.find('div', class_='inner desktop')
desktopControls = innerDesktop.find('div', class_='desktop controls')
controlSelect = desktopControls.find_all('div', class_='control select')
divisionFilter = controlSelect[2]
womenInput = divisionFilter.find('div', class_='input')
findSelect = womenInput.find('select')
items = findSelect.select('option')
she = items[1]
femaleDiv = she.get('value')

In [9]:
# loading womens results
womens = f'/leaderboard/games/2021?division={femaleDiv}&sort=0'
webpage = f'{url}{womens}'
browser.visit(webpage)
leftArrow = browser.find_by_xpath('//button[starts-with(@class,"workout")]')
leftArrow.click()
leftArrow.click()

In [10]:
# Retrieve elements (womens)

# re-defining page right arrow
rightArrow = browser.find_by_xpath('//button[contains(@class,"right")]')

# initializing lists
overall = []
individual = []
x = 0
y = 0

# find table row to iterate through
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
articles = soup.find('tbody')
athletes = articles.find_all('tr', class_='collapsed')

# for loop to find women's info
for i in athletes:
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find('tbody')
    athletes = articles.find_all('tr', class_='collapsed')
    athleteJustinM = athletes[x]
    fName = athleteJustinM.find('div', class_='first-name').text.strip()
    individual.append(fName)
    lName = athleteJustinM.find('div', class_='last-name').text.strip()
    individual.append(lName)
    #new
    fullName = f'{fName} {lName}'
    individual.append(fullName)
    
    a = athleteJustinM.find('a')
    href = a.get('href')
    
    webpage = f'{url}{href}'
    browser.visit(webpage)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    body = soup.find('body')
    conatinerOverlay = body.find('div', class_='container-overlay')
    mainContent = conatinerOverlay.find('div', class_='main-content')

    md9 = mainContent.find('div', class_='col-md-9')
    img = md9.find('img')
    picture = img.get('src')

    pageCover = mainContent.find('div', class_='page-cover')
    
    overlayed = pageCover.find('div', class_='overlayed')
    clearfix = overlayed.find('ul')
    allLi = clearfix.find_all('li')
    selectedLi = allLi[y]
    flagIMG = selectedLi.find('img')
    flag = flagIMG.get('src')
    flagURL = f'https:{flag}'

    selectedLi = allLi[y]
    countryDiv = selectedLi.find('div', class_='text')
    country = countryDiv.find('small').text

    thirdRow = clearfix.find('li', class_='third-row')
    gymText = thirdRow.find('div', class_='text')
    try:
        affiliate = gymText.find('a').text
    except:
        affiliate = ""

    individual.append(picture)
    individual.append(country)
    individual.append(flagURL)
    individual.append(affiliate)

    browser.back()
    rightArrow = browser.find_by_xpath('//button[contains(@class,"right")]')
    leftArrow = browser.find_by_xpath('//button[starts-with(@class,"workout")]')
    leftArrow.click()
    leftArrow.click()
    #new fin
    scores = athleteJustinM.find_all('span', class_='result')
    try:
        individual.append(scores[0].text.strip())
        individual.append(scores[1].text.strip())
        individual.append(scores[2].text.strip())
        individual.append(scores[3].text.strip())
        individual.append(scores[4].text.strip())
        individual.append(scores[5].text.strip())
    except:
        print(fName, "no score")
        pass

    rightArrow.click()

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find('tbody')
    athletes = articles.find_all('tr', class_='collapsed')
    athleteJustinM = athletes[x]
    scores = athleteJustinM.find_all('span', class_='result')
    try:
        individual.append(scores[0].text.strip())
        individual.append(scores[1].text.strip())
        individual.append(scores[2].text.strip())
        individual.append(scores[3].text.strip())
        individual.append(scores[4].text.strip())
        individual.append(scores[5].text.strip())
    except:
        print(fName, "no score 2")
        pass

    rightArrow.click()

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find('tbody')
    athletes = articles.find_all('tr', class_='collapsed')
    athleteJustinM = athletes[x]
    scores = athleteJustinM.find_all('span', class_='result')
    try:
        individual.append(scores[3].text.strip())
        individual.append(scores[4].text.strip())
        individual.append(scores[5].text.strip())
    except:
        print(fName, "no score 3")
        pass

    overall.append(individual)
    leftArrow.click()
    leftArrow.click()
    x+=1
    time.sleep(.4)
#     break

Svetlana no score
Svetlana no score 2
Svetlana no score 3
Bethany no score
Bethany no score 2
Bethany no score 3
Larissa no score
Larissa no score 2
Larissa no score 3


In [11]:
# pulling the 36 female athletes that competed (40 made it, 4 withdrew prior to event)
k = 0
s = 22
allAthletesF = []
for i in individual:
    allAthletesF.append(individual[k:s])
    k+=22
    s+=22
    
athlete_listF = allAthletesF[:36]

In [12]:
# putting female athletes into a dataframe
full_dfF = pd.DataFrame(athlete_listF)

In [13]:
# this process will visit multiple pages parsing through the website to find the scoring table
# after finding scoring table, it turned out to be an image so I saved that locally

main_nav = body.find('ul')
mainNavContainer = body.find('div', class_='slick-track')
activeClass = mainNavContainer.find('li', class_='active')
activeClassHref = activeClass.find('a').get('href')

webpage = f'{url}{activeClassHref}'
browser.visit(webpage)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
body = soup.find('body')
containerOverlay = body.find('div', class_='container-overlay')
mainContent = containerOverlay.find('div', class_='main-content')
pageSection = mainContent.find_all('div', class_='page-section')[1]
h3 = pageSection.find('h3')
annoucements = h3.find('a').get('href')

webpage = f'{url}{annoucements}'
browser.visit(webpage)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
body = soup.find('body')
containerOverlay = body.find('div', class_='container-overlay')
mainContent = containerOverlay.find('div', class_='main-content')
ul = mainContent.find('ul', class_='announcements-list')
li = ul.find_all('li')[26]
scoring = li.find('a').get('href')

webpage = f'{url}{scoring}'
browser.visit(webpage)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
body = soup.find('body')
containerOverlay = body.find('div', class_='container-overlay')
mainContent = containerOverlay.find('div', class_='main-content')
scoringImage = mainContent.find('img').get('src')

webpage = f'{scoringImage}'
browser.visit(webpage)

In [14]:
# Close browser
browser.quit()

In [15]:
# merging the male and female dataframes vertically / defining columns
allAlthlese_df = full_df.append(full_dfF, ignore_index=True)
allAlthlese_df.columns = ['first_name', 'last_name', 'full_name', 'image', 'country', 'flag', 'affiliate',
                          'event_1','event_2', 'event_3', 'event_4', 'event_5',
                          'event_6','event_7', 'event_8', 'event_9', 'event_10',
                          'event_11', 'event_12', 'event_13', 'event_14', 'event_15']

# replacing characters
allAlthlese_df['event_1'] = allAlthlese_df['event_1'].str.replace('(', '')
allAlthlese_df['event_1'] = allAlthlese_df['event_1'].str.replace(')', '')
allAlthlese_df['event_2'] = allAlthlese_df['event_2'].str.replace('(', '')
allAlthlese_df['event_2'] = allAlthlese_df['event_2'].str.replace(')', '')
allAlthlese_df['event_3'] = allAlthlese_df['event_3'].str.replace('(', '')
allAlthlese_df['event_3'] = allAlthlese_df['event_3'].str.replace(')', '')
allAlthlese_df['event_4'] = allAlthlese_df['event_4'].str.replace('(', '')
allAlthlese_df['event_4'] = allAlthlese_df['event_4'].str.replace(')', '')
allAlthlese_df['event_5'] = allAlthlese_df['event_5'].str.replace('(', '')
allAlthlese_df['event_5'] = allAlthlese_df['event_5'].str.replace(')', '')
allAlthlese_df['event_6'] = allAlthlese_df['event_6'].str.replace('(', '')
allAlthlese_df['event_6'] = allAlthlese_df['event_6'].str.replace(')', '')
allAlthlese_df['event_7'] = allAlthlese_df['event_7'].str.replace('(', '')
allAlthlese_df['event_7'] = allAlthlese_df['event_7'].str.replace(')', '')
allAlthlese_df['event_8'] = allAlthlese_df['event_8'].str.replace('(', '')
allAlthlese_df['event_8'] = allAlthlese_df['event_8'].str.replace(')', '')
allAlthlese_df['event_9'] = allAlthlese_df['event_9'].str.replace('(', '')
allAlthlese_df['event_9'] = allAlthlese_df['event_9'].str.replace(')', '')
allAlthlese_df['event_10'] = allAlthlese_df['event_10'].str.replace('(', '')
allAlthlese_df['event_10'] = allAlthlese_df['event_10'].str.replace(')', '')
allAlthlese_df['event_11'] = allAlthlese_df['event_11'].str.replace('(', '')
allAlthlese_df['event_11'] = allAlthlese_df['event_11'].str.replace(')', '')
allAlthlese_df['event_11'] = allAlthlese_df['event_11'].str.replace(' rep', '')
allAlthlese_df['event_12'] = allAlthlese_df['event_12'].str.replace('(', '')
allAlthlese_df['event_12'] = allAlthlese_df['event_12'].str.replace(')', '')
allAlthlese_df['event_12'] = allAlthlese_df['event_12'].str.replace(' lb', '')
allAlthlese_df['event_13'] = allAlthlese_df['event_13'].str.replace('(', '')
allAlthlese_df['event_13'] = allAlthlese_df['event_13'].str.replace(')', '')
allAlthlese_df['event_14'] = allAlthlese_df['event_14'].str.replace('(', '')
allAlthlese_df['event_14'] = allAlthlese_df['event_14'].str.replace(')', '')
allAlthlese_df['event_15'] = allAlthlese_df['event_15'].str.replace('(', '')
allAlthlese_df['event_15'] = allAlthlese_df['event_15'].str.replace(')', '')

In [16]:
# previously saved to csv - opened in Microsoft Excel to make a ranking and noticed some CAP's
# needed to be changed to work with rank

# replacing 'WD' and '--' to blank
allAlthlese_df = allAlthlese_df.replace({'WD': '', '--': ''}, regex=True)

# appending zero's to some CAP numbers to work with ranking
allAlthlese_df.replace(to_replace = "CAP", value = "CAP+00", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+1", value = "CAP+01", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+2", value = "CAP+02", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+3", value = "CAP+03", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+4", value = "CAP+04", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+5", value = "CAP+05", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+6", value = "CAP+06", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+7", value = "CAP+07", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+8", value = "CAP+08", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+9", value = "CAP+09", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+54", value = "CAP+054", inplace = True)
allAlthlese_df.replace(to_replace = "CAP+73", value = "CAP+073", inplace = True)

In [17]:
# ranking event columns and creating their own columns
allAlthlese_df['event_1_rank'] = allAlthlese_df['event_1'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_2_rank'] = allAlthlese_df['event_2'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_3_rank'] = allAlthlese_df['event_3'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_4_rank'] = allAlthlese_df['event_4'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_5_rank'] = allAlthlese_df['event_5'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_6_rank'] = allAlthlese_df['event_6'].rank(axis=0,method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_7_rank'] = allAlthlese_df['event_7'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_8_rank'] = allAlthlese_df['event_8'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_9_rank'] = allAlthlese_df['event_9'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_10_rank'] = allAlthlese_df['event_10'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_11_rank'] = allAlthlese_df['event_11'].rank(method='min', na_option='bottom', ascending=False)
allAlthlese_df['event_12_rank'] = allAlthlese_df['event_12'].rank(method='min', na_option='bottom', ascending=False)
allAlthlese_df['event_13_rank'] = allAlthlese_df['event_13'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_14_rank'] = allAlthlese_df['event_14'].rank(method='min', na_option='bottom', ascending=True)
allAlthlese_df['event_15_rank'] = allAlthlese_df['event_15'].rank(method='min', na_option='bottom', ascending=True)

In [18]:
#preview dataframe
allAlthlese_df

,first_name,last_name,full_name,image,country,flag,affiliate,event_1,event_2,event_3,...,event_6_rank,event_7_rank,event_8_rank,event_9_rank,event_10_rank,event_11_rank,event_12_rank,event_13_rank,event_14_rank,event_15_rank
0,Justin,Medeiros,Justin Medeiros,https://profilepicsbucket.crossfit.com/4aa64-P...,United States,https://assets.crossfit.com/build/img/sites/ga...,CrossFit Fort Vancouver,1:10:54.31,09:13.54,01:18.21,...,13.0,20.0,11.0,15.0,22.0,11.0,4.0,40.0,40.0,37.0
1,Patrick,Vellner,Patrick Vellner,https://profilepicsbucket.crossfit.com/d471c-P...,Canada,https://assets.crossfit.com/build/img/sites/ga...,CrossFit Nanaimo,1:19:48.62,07:42.42,01:15.72,...,41.0,28.0,36.0,11.0,30.0,5.0,3.0,38.0,39.0,50.0
2,Brent,Fikowski,Brent Fikowski,https://profilepicsbucket.crossfit.com/93ab7-P...,Canada,https://assets.crossfit.com/build/img/sites/ga...,,1:12:08.38,08:35.34,01:18.25,...,41.0,35.0,29.0,7.0,52.0,7.0,4.0,55.0,53.0,57.0
3,Björgvin Karl,Guðmundsson,Björgvin Karl Guðmundsson,https://profilepicsbucket.crossfit.com/4c5dc-P...,Iceland,https://assets.crossfit.com/build/img/sites/ga...,CrossFit Hengill,1:11:10.13,08:45.81,01:17.68,...,41.0,54.0,15.0,21.0,34.0,3.0,11.0,37.0,46.0,61.0
4,Saxon,Panchik,Saxon Panchik,https://profilepicsbucket.crossfit.com/00087-P...,United States,https://assets.crossfit.com/build/img/sites/ga...,CrossFit Cliffside,1:14:00.84,09:11.03,01:15.94,...,12.0,16.0,17.0,38.0,38.0,16.0,4.0,73.0,42.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Michelle,Basnett,Michelle Basnett,https://profilepicsbucket.crossfit.com/3651c-P...,South Africa,https://assets.crossfit.com/build/img/sites/ga...,CrossFit Cornelius,1:14:38.06,11:45.77,01:42.35,...,41.0,48.0,48.0,67.0,1.0,41.0,41.0,1.0,1.0,1.0
70,Caroline,Conners,Caroline Conners,https://profilepicsbucket.crossfit.com/93573-P...,United States,https://assets.crossfit.com/build/img/sites/ga...,CrossFit MF,1:26:41.53,CAP+06,01:36.77,...,37.0,36.0,59.0,71.0,1.0,41.0,41.0,1.0,1.0,1.0
71,Seungyeon,Choi,Seungyeon Choi,https://profilepicsbucket.crossfit.com/e8e28-P...,"Korea, Republic of",https://assets.crossfit.com/build/img/sites/ga...,CrossFit Limelight,1:43:14.37,CAP+12,01:38.52,...,41.0,57.0,72.0,53.0,1.0,41.0,41.0,1.0,1.0,1.0
72,Dani,Speegle,Dani Speegle,https://profilepicsbucket.crossfit.com/47a8d-P...,United States,https://assets.crossfit.com/build/img/sites/ga...,CrossFit Invictus,1:17:37.12,11:06.00,01:29.13,...,1.0,1.0,1.0,1.0,1.0,41.0,41.0,1.0,1.0,1.0


In [19]:
# save locally to CSV
allAlthlese_df.to_csv('crossfit.csv', index=False)

In [20]:
# stop program run timer
stop = timeit.default_timer()

# view how long it took to run
# try and find improvments to speed up later
print('Time to run:', stop - start)

Time to run: 442.342311845
